In [188]:
import pandas as pd
pd.read_csv('facebook_web_log.csv')
df = pd.read_csv('facebook_web_log.csv')
df

,user_id,date,time,action
0,0,4/25/2019,13:30:15,page_load
1,0,4/25/2019,13:30:18,page_load
2,0,4/25/2019,13:30:40,scroll_down
3,0,4/25/2019,13:30:45,scroll_up
4,0,4/25/2019,13:31:10,scroll_down
5,0,4/25/2019,13:31:25,scroll_down
6,0,4/25/2019,13:31:40,page_exit
7,1,4/25/2019,13:40:00,page_load
8,1,4/25/2019,13:40:10,scroll_down
9,1,4/25/2019,13:40:15,scroll_down


Calculate each user's average session time. A session is defined as the time difference between a page_load and page_exit. For simplicity, assume a user has only 1 session per day and if there are multiple of the same events on that day, consider only the latest page_load and earliest page_exit. Output the user_id and their average session time.

In [189]:
df['time'].apply(pd.to_datetime)
df['time']=df['time'].apply(pd.to_datetime).dt.time
df['date'].apply(pd.to_datetime)
df['date']=df['date'].apply(pd.to_datetime)

In [190]:
# Change type of action column
df['action'] = df['action'].apply(str)
df

,user_id,date,time,action
0,0,2019-04-25,13:30:15,page_load
1,0,2019-04-25,13:30:18,page_load
2,0,2019-04-25,13:30:40,scroll_down
3,0,2019-04-25,13:30:45,scroll_up
4,0,2019-04-25,13:31:10,scroll_down
5,0,2019-04-25,13:31:25,scroll_down
6,0,2019-04-25,13:31:40,page_exit
7,1,2019-04-25,13:40:00,page_load
8,1,2019-04-25,13:40:10,scroll_down
9,1,2019-04-25,13:40:15,scroll_down


## the latest page_load and earliest page_exit

In [201]:
# latest page_load = max time of each date
page_load = df[df['action'].str.contains('page_load')]
page_load.groupby(['user_id','date'])['time'].max().reset_index()
page_load.drop(columns=['action'])

,user_id,date,time
0,0,2019-04-25,13:30:15
1,0,2019-04-25,13:30:18
7,1,2019-04-25,13:40:00
14,2,2019-04-25,13:41:21
18,1,2019-04-26,11:15:00
23,0,2019-04-28,14:30:15
24,0,2019-04-28,14:30:10


In [202]:
# earliest page_exit = min time of each date
page_exit = df[df['action'].str.contains('page_exit')]
page_exit.groupby(['user_id','date'])['time'].min().reset_index()
page_exit.drop(columns=['action'])

,user_id,date,time
6,0,2019-04-25,13:31:40
13,1,2019-04-25,13:40:35
22,1,2019-04-26,11:15:35
26,0,2019-04-28,15:31:40


In [204]:
pd.merge(page_load,page_exit,left_on=['user_id','date'],right_on=['user_id','date'])

,user_id,date,time_x,action_x,time_y,action_y
0,0,2019-04-25,13:30:15,page_load,13:31:40,page_exit
1,0,2019-04-25,13:30:18,page_load,13:31:40,page_exit
2,1,2019-04-25,13:40:00,page_load,13:40:35,page_exit
3,1,2019-04-26,11:15:00,page_load,11:15:35,page_exit
4,0,2019-04-28,14:30:15,page_load,15:31:40,page_exit
5,0,2019-04-28,14:30:10,page_load,15:31:40,page_exit


In [222]:
q = pd.merge(page_load,page_exit,left_on=['user_id','date'],right_on=['user_id','date']).rename(columns={'time_x':'load_time','time_y':'exit_time'})
q

,user_id,date,load_time,action_x,exit_time,action_y
0,0,2019-04-25,13:30:15,page_load,13:31:40,page_exit
1,0,2019-04-25,13:30:18,page_load,13:31:40,page_exit
2,1,2019-04-25,13:40:00,page_load,13:40:35,page_exit
3,1,2019-04-26,11:15:00,page_load,11:15:35,page_exit
4,0,2019-04-28,14:30:15,page_load,15:31:40,page_exit
5,0,2019-04-28,14:30:10,page_load,15:31:40,page_exit


In [228]:
# errors to change object into date time: <class 'datetime.time'> is not convertible to datetime
# instead apply(pd.to_datetime) we must use: pd.Timestamp
# instead apply(pd.Timestamp) we must use: pd.to_datetime(df['col'].astype(str))
q['load_time']=pd.to_datetime(q['load_time'].astype(str))
q['exit_time']=pd.to_datetime(q['exit_time'].astype(str))

In [239]:
q['session_time'] = q['exit_time']-q['load_time']
q

,user_id,date,load_time,action_x,exit_time,action_y,session_time
0,0,2019-04-25,2023-01-11 13:30:15,page_load,2023-01-11 13:31:40,page_exit,0 days 00:01:25
1,0,2019-04-25,2023-01-11 13:30:18,page_load,2023-01-11 13:31:40,page_exit,0 days 00:01:22
2,1,2019-04-25,2023-01-11 13:40:00,page_load,2023-01-11 13:40:35,page_exit,0 days 00:00:35
3,1,2019-04-26,2023-01-11 11:15:00,page_load,2023-01-11 11:15:35,page_exit,0 days 00:00:35
4,0,2019-04-28,2023-01-11 14:30:15,page_load,2023-01-11 15:31:40,page_exit,0 days 01:01:25
5,0,2019-04-28,2023-01-11 14:30:10,page_load,2023-01-11 15:31:40,page_exit,0 days 01:01:30
